In [69]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
from statsmodels.graphics.mosaicplot import mosaic
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold

In [70]:
df = pd.read_csv("../dataset_diabetes/diabetic_data.csv")

In [87]:
knn_data = df.drop(['encounter_id','patient_nbr', 'weight', 'examide', 'citoglipton','diag_1','diag_2','diag_3'],axis=1)
#change ? and None to Nan
data_clean = knn_data.replace('?', np.nan)
data_clean = data_clean.replace('None', np.nan)
data_clean = data_clean.drop(['payer_code','medical_specialty','max_glu_serum','A1Cresult'],axis=1)
#numerical columns: Time in hospital,Number of lab procedures,Number of procedures,
#Number of medications,Number of outpatient visits,Number of emergency visits,
#Number of inpatient visits,Number of diagnoses,

numerical = ['time_in_hospital','num_lab_procedures','num_procedures','num_medications',
             'number_outpatient','number_emergency','number_inpatient',
             'number_diagnoses','admission_type_id','discharge_disposition_id','admission_source_id']

numerical_df = data_clean[numerical]
#want just the Y's
output_var = data_clean['readmitted']  
non_numerical_df = data_clean.drop(numerical,axis=1)
non_numerical_df = non_numerical_df.drop(['readmitted'],axis=1)
df1 = pd.DataFrame()
df2 = pd.DataFrame()
for i in non_numerical_df.columns:
    if (len(set(non_numerical_df[i])) > 2): #if set is greater than 2, want to get a separate df
        df1[i] = non_numerical_df[i]
    else: #if set is less than 2, want another df
        df2[i] = non_numerical_df[i]
dummy2 = pd.get_dummies(df2) #change to dummy variables
dummy1 = pd.get_dummies(df1)
frames = [numerical_df,dummy1,dummy2,output_var]
cleaned_df = pd.concat(frames,axis=1)

cleaned_df['readmitted'] = cleaned_df['readmitted'].map( {'NO': 0, '>30': 1, '<30': 1} ).astype(int)

# Create a new column that for each row, generates a random number between 0 and 1, and
# if that value is less than or equal to .75, then sets the value of that cell as True
# and false otherwise. This is a quick and dirty way of randomly assigning some rows to
# be used as the training data and some as the test data.
cleaned_df['is_train'] = np.random.uniform(0, 1, len(cleaned_df)) <= .75
# Create two new dataframes, one with the training rows, one with the test rows
train, test = cleaned_df[cleaned_df['is_train']==True], cleaned_df[cleaned_df['is_train']==False]
# Create a list of the feature column's names
features = cleaned_df.columns[:-2]
#y = pd.factorize(train['readmitted'])[0]
y = train['readmitted']

In [88]:
train_clean = train.drop(['is_train'],axis=1)
test_clean = test.drop(['is_train'],axis=1)

In [89]:
train_clean.values

array([[ 1, 41,  0, ...,  1,  0,  0],
       [ 2, 11,  5, ...,  0,  1,  0],
       [ 2, 44,  1, ...,  0,  1,  0],
       ..., 
       [ 5, 33,  3, ...,  0,  1,  0],
       [ 1, 53,  0, ...,  0,  1,  0],
       [ 6, 13,  3, ...,  1,  0,  0]])

In [90]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train) #fit

    def predict(self, x):
        return self.clf.predict(x) #predict
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [91]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return(oof_train.reshape(-1, 1), oof_test.reshape(-1, 1))

In [92]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [93]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [94]:
train.head(3)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,admission_type_id,discharge_disposition_id,...,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted,is_train
0,1,41,0,1,0,0,0,1,6,25,...,1,0,1,0,0,1,1,0,0,True
2,2,11,5,13,2,0,1,6,1,1,...,1,0,1,0,0,1,0,1,0,True
3,2,44,1,16,0,0,0,7,1,1,...,1,0,1,0,1,0,0,1,0,True


In [96]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['readmitted'].ravel()
features = cleaned_df.columns[:-2]
train = train_clean[features]
test = test_clean[features]
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

In [97]:
x_train

array([[ 1, 41,  0, ...,  1,  1,  0],
       [ 2, 11,  5, ...,  1,  0,  1],
       [ 2, 44,  1, ...,  0,  0,  1],
       ..., 
       [ 5, 33,  3, ...,  1,  0,  1],
       [ 1, 53,  0, ...,  0,  0,  1],
       [ 6, 13,  3, ...,  1,  1,  0]])

In [98]:
x_test

array([[ 3, 59,  0, ...,  0,  0,  1],
       [ 1, 51,  0, ...,  0,  0,  1],
       [ 9, 47,  2, ...,  1,  0,  1],
       ..., 
       [ 2, 46,  6, ...,  1,  0,  1],
       [ 1,  1,  0, ...,  0,  0,  1],
       [10, 45,  2, ...,  0,  0,  1]])

In [99]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [100]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
#rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
#ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
#gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
#svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

Training is complete
